#### Environment: pt_fpad  
Python: 3.10.4     
Pytorch: 2.1.1+cu118

In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torchvision import transforms, models
import torch.optim as optim
import os
import random
import cv2
import numpy as np

In [2]:
class_label_real = 0
class_label_attack = 1

Replay Mobile Dataset:

Training -> 312 (Real 120, Attack 192) ->  312/1030 * 100 = 30.29%  
Validation -> 416 (Real 160, Attack 256) -> 416/1030 * 100 = 40.38%  
Testing -> 302 (Real 110, Attack 192) -> 302/1030 * 100 = 29.32%  
Total = 1030  

Replay Attack Dataset:  
Training -> 360 (Real 60, Attack Fixed 150, Attack Hand 150) ->  360/1200 * 100 = 30%  
Validation -> 360 (Real 60, Attack Fixed 150, Attack Hand 150) -> 360/1200 * 100 = 30%  
Testing -> 480 (Real 80, Attack Fixed 200, Attack Hand 200) -> 480/1200 * 100 = 40%  
Total = 1200  

Rose-Youtu Dataset:  
Training -> 1397 (Real 358, Attack 1039) -> 1397/3495 * 100 = 40%  
Validation -> 350 (Real 90, Attack 260) -> 350/3495 * 100 = 10%  
Testing -> 1748 (Real 449, Attack 1299) -> 1748/3495 * 100 = 50%   
Total = 3495




In [3]:
# Example
# Provide dataset path for Replay Mobile and Rose Youtu Datasets e.g 
# data_path_train_real = '/home/taha/FASdatasets/Sohail/Rose_Youtu/train/real'
# data_path_train_attack = '/home/taha/FASdatasets/Sohail/Rose_Youtu/train/attack'

# For Replay Attack Dataset e.g.
# data_path_train_real = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/real'
# data_path_train_attack_fixed = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/attack/fixed'
# data_path_train_attack_hand = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/attack/hand'

# Follow the same above steps for Validation and Test Datasets

data_path_train_real = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/real'
# data_path_train_attack = '/home/taha/FASdatasets/Sohail/Rose_Youtu/train/attack'
data_path_train_attack_fixed = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/attack/fixed'
data_path_train_attack_hand = '/home/taha/FASdatasets/Sohail/Replay_Attack/train/attack/hand'

data_path_devel_real = '/home/taha/FASdatasets/Sohail/Replay_Attack/devel/real'
# data_path_devel_attack = '/home/taha/FASdatasets/Sohail/Rose_Youtu/devel/attack'
data_path_devel_attack_fixed = '/home/taha/FASdatasets/Sohail/Replay_Attack/devel/attack/fixed'
data_path_devel_attack_hand = '/home/taha/FASdatasets/Sohail/Replay_Attack/devel/attack/hand'

data_path_test_real = '/home/taha/FASdatasets/Sohail/Replay_Attack/test/real'
# data_path_test_attack = '/home/taha/FASdatasets/Sohail/Rose_Youtu/test/attack'
data_path_test_attack_fixed = '/home/taha/FASdatasets/Sohail/Replay_Attack/test/attack/fixed'
data_path_test_attack_hand = '/home/taha/FASdatasets/Sohail/Replay_Attack/test/attack/hand'

In [4]:
def load_samples(path, class_label, transform): #Select N frames returned from read_all_frames and assign labels to all samples of same class
        frames = read_all_frames(path)
        total_frames = list(range(0, frames.shape[0], 1))
        selected_samples = random.sample(total_frames, 1)
        samples =[]
        # Assign the same class label to all samples
        label = class_label
        samples =(transform(frames[selected_samples].squeeze()), label)     
        return samples

def read_all_frames(video_path): # reads all frames from a particular video and converts them to PyTorch tensors.
    frame_list = []
    video = cv2.VideoCapture(video_path)
    success = True
    while success:
        success, frame = video.read()
        if success:
            frame = cv2.resize(frame, (224, 224), interpolation=cv2.INTER_AREA) #framesize kept 224,224
            frame_list.append(frame)
    frame_list = np.array(frame_list)
    return frame_list

class VideoDataset(Dataset):
    def __init__(self, data_path, class_label):
        self.data_path = data_path #path for directory containing video files
        self.video_files = [file for file in os.listdir(data_path) if file.endswith('.mov') or file.endswith('.mp4')] #list of video files in the specified directory #.mov for RA and RM, .mp4 for RY
        self.class_label = class_label #manually assign class_label for your desired class while loading
        self.data_length = len(self.video_files) 
        self.transform = transforms.Compose([transforms.ToTensor()])

    def __len__(self): # returns the total number of samples in the dataset
        return self.data_length

    def __getitem__(self, idx): # loads and returns a sample from the dataset at the given index
        file = self.video_files[idx]
        path = os.path.join(self.data_path, file)
        frames= load_samples(path, self.class_label, self.transform)

        return frames

In [5]:
# Create datasets for Replay Attack, Replay Mobile and Rose Youtu Datasets

train_dataset_real = VideoDataset(data_path_train_real, class_label_real)
# train_dataset_attack = VideoDataset(data_path_train_attack, class_label_attack)

# For Replay Attack Dataset
train_dataset_attack_fixed = VideoDataset(data_path_train_attack_fixed, class_label_attack)
train_dataset_attack_hand = VideoDataset(data_path_train_attack_hand, class_label_attack)
# Same steps follow for Validation and Test Datasets

val_dataset_real = VideoDataset(data_path_devel_real, class_label_real)
# val_dataset_attack = VideoDataset(data_path_devel_attack, class_label_attack)
val_dataset_attack_fixed = VideoDataset(data_path_devel_attack_fixed, class_label_attack)
val_dataset_attack_hand = VideoDataset(data_path_devel_attack_hand, class_label_attack)

test_dataset_real = VideoDataset(data_path_test_real, class_label_real)
# test_dataset_attack = VideoDataset(data_path_test_attack, class_label_attack)
test_dataset_attack_fixed = VideoDataset(data_path_test_attack_fixed, class_label_attack)
test_dataset_attack_hand = VideoDataset(data_path_test_attack_hand, class_label_attack)

In [6]:
# Create DataLoader for Replay Mobile and Rose Youtu Datasets

train_loader_real = DataLoader(train_dataset_real, batch_size=1, shuffle=True)
# train_loader_attack = DataLoader(train_dataset_attack, batch_size=1, shuffle=True)

# For Replay Attack Dataset
train_loader_attack_fixed = DataLoader(train_dataset_attack_fixed, batch_size=1, shuffle=True)
train_loader_attack_hand = DataLoader(train_dataset_attack_hand, batch_size=1, shuffle=True)
# Same steps follow for Validation and Test DataLoader

val_loader_real = DataLoader(val_dataset_real, batch_size=1, shuffle=False)
# val_loader_attack = DataLoader(val_dataset_attack, batch_size=1, shuffle=False)
val_loader_attack_fixed = DataLoader(val_dataset_attack_fixed, batch_size=1, shuffle=False)
val_loader_attack_hand = DataLoader(val_dataset_attack_hand, batch_size=1, shuffle=False)

test_loader_real = DataLoader(test_dataset_real, batch_size=1, shuffle=False)
# test_loader_attack = DataLoader(test_dataset_attack, batch_size=1, shuffle=False)
test_loader_attack_fixed = DataLoader(test_dataset_attack_fixed, batch_size=1, shuffle=False)
test_loader_attack_hand = DataLoader(test_dataset_attack_hand, batch_size=1, shuffle=False)

In [7]:
# Concatenate datasets for Replay Mobile and Rose Youtu Datasets
# concatenated_train_dataset = ConcatDataset([train_dataset_real, train_dataset_attack])
# concatenated_val_dataset = ConcatDataset([val_dataset_real, val_dataset_attack])
# concatenated_test_dataset = ConcatDataset([test_dataset_real, test_dataset_attack])

# Concatenate datasets for Replay Attack Dataset
concatenated_train_dataset = ConcatDataset([train_dataset_real, train_dataset_attack_fixed, train_dataset_attack_hand])
concatenated_val_dataset = ConcatDataset([val_dataset_real, val_dataset_attack_fixed, val_dataset_attack_hand])
concatenated_test_dataset = ConcatDataset([test_dataset_real, test_dataset_attack_fixed, test_dataset_attack_hand])

concatenated_train_loader = DataLoader(concatenated_train_dataset, batch_size=64, shuffle=True, pin_memory=False, num_workers=8)
concatenated_val_loader = DataLoader(concatenated_val_dataset, batch_size=64, shuffle=False, pin_memory=False, num_workers=8)
concatenated_test_loader = DataLoader(concatenated_test_dataset, batch_size=64, shuffle=False, pin_memory=False, num_workers=8)

In [8]:
# Print dataset sizes
print(f"Training set size: {len(concatenated_train_dataset)}")
print(f"Validation set size: {len(concatenated_val_dataset)}")
print(f"Test set size: {len(concatenated_test_dataset)}")

Training set size: 360
Validation set size: 360
Test set size: 480
